In [4]:
import json
import numpy as np
import pandas as pd

from collections import Counter
from sklearn.metrics import f1_score

from sys import path

path.append('../utils/')

from utils import get_datasets

from utils import load_preds

from scipy.stats import t as table_t

In [5]:
DATASETS = ["20ng", "acm", "webkb", "reut"]
DATASETS = ["acm", "20ng"]

In [6]:
pd_datasets = get_datasets(DATASETS)

In [10]:
st1 = "calibrated"

CLFS = [
["bert", "normal_probas"],
["kfr", "normal_probas"],
["ktmk", "normal_probas"],
["ktr", "normal_probas"],
["lfr", "normal_probas"],
["lsfr", "normal_probas"],
["lstmk", "normal_probas"],
["lstr", "normal_probas"],
["ltr", "normal_probas"],
["xlnet", "normal_probas"]
]

posfix = '/'.join(sorted([ f"{c[0]}_{c[1]}" for c in CLFS ]))
st2s = [
    f"local_rf/zero_train_False/{alfa}/{posfix}/dist_probas_probas_based"
    for alfa in [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
]

#st2s = ["calibrated"]

fold_values = {}
for dataset in DATASETS:
    fold_values[dataset] = {}
    for st2 in st2s:
        fold_values[dataset][st2] = []
        diffs = []
        alpha = 0.05 / 2
        for fold in np.arange(10):

            ppath = f"/home/welton/data/stacking/stacking_output/{dataset}/10_folds/logistic_regression/{st1}/fold_{fold}/scoring.json"
            with open(ppath, 'r') as fd:
                st1_macro = json.load(fd)["f1_macro"]
            
            mpath = f"/home/welton/data/stacking/stacking_output/{dataset}/10_folds/logistic_regression/{st2}/fold_{fold}/scoring.json"
            with open(mpath, 'r') as fd:
                st2_macro = json.load(fd)["f1_macro"]
            
            diffs.append(st2_macro - st1_macro)
            fold_values[dataset][st2].append([st1_macro, st2_macro])
        #    print(st2_macro, st1_macro, st2_macro - st1_macro)
            
        mean = np.mean(diffs)
        #print(mean)
        std = np.std(diffs) / np.sqrt(len(diffs))

        t = table_t.ppf(1 - alpha, df=9)
        ground = np.round(mean - t * std, decimals=4)
        ceiling = np.round(mean + t * std, decimals=4)

        print(f"{dataset.upper()} - ({ground} - {ceiling})")
        #print("*"*100)

ACM - (-0.1245 - -0.0943)
ACM - (-0.1245 - -0.0954)
ACM - (-0.1252 - -0.096)
ACM - (-0.126 - -0.0968)
ACM - (-0.1261 - -0.0965)
ACM - (-0.1262 - -0.0966)
20NG - (-0.0927 - -0.0814)
20NG - (-0.0947 - -0.0831)
20NG - (-0.094 - -0.0769)
20NG - (-0.0943 - -0.0822)
20NG - (-0.0959 - -0.0804)
20NG - (-0.0945 - -0.0794)


In [11]:
for dataset in DATASETS:
    for st2 in st2s:
        df = pd.DataFrame(fold_values[dataset][st2], columns=["st1", "st2"])
        mrep = np.round(np.mean(df.st1.values) * 100, decimals=2)
        prob = np.round(np.mean(df.st2.values) * 100, decimals=2)
        print(f"{dataset.upper()} - {st1}: {mrep} {st2}: {prob}")

ACM - calibrated: 73.95 local_rf/zero_train_False/0.4/bert_normal_probas/kfr_normal_probas/ktmk_normal_probas/ktr_normal_probas/lfr_normal_probas/lsfr_normal_probas/lstmk_normal_probas/lstr_normal_probas/ltr_normal_probas/xlnet_normal_probas/dist_probas_probas_based: 63.01
ACM - calibrated: 73.95 local_rf/zero_train_False/0.5/bert_normal_probas/kfr_normal_probas/ktmk_normal_probas/ktr_normal_probas/lfr_normal_probas/lsfr_normal_probas/lstmk_normal_probas/lstr_normal_probas/ltr_normal_probas/xlnet_normal_probas/dist_probas_probas_based: 62.96
ACM - calibrated: 73.95 local_rf/zero_train_False/0.6/bert_normal_probas/kfr_normal_probas/ktmk_normal_probas/ktr_normal_probas/lfr_normal_probas/lsfr_normal_probas/lstmk_normal_probas/lstr_normal_probas/ltr_normal_probas/xlnet_normal_probas/dist_probas_probas_based: 62.89
ACM - calibrated: 73.95 local_rf/zero_train_False/0.7/bert_normal_probas/kfr_normal_probas/ktmk_normal_probas/ktr_normal_probas/lfr_normal_probas/lsfr_normal_probas/lstmk_normal_

In [25]:
np.load("/home/welton/data/oracle/local_rf/isotonic/acm/10_folds/ktmk/dist_probas_probas_based/0/train.npz")['y']

array([0., 1., 1., ..., 0., 1., 0.])